IO在计算机中指Input/Output，也就是输入和输出。由于程序和运行时数据是在内存中驻留，由CPU这个超快的计算核心来执行，涉及到数据交换的地方，通常是磁盘、网络等，就需要IO接口。

比如你打开浏览器，访问新浪首页，浏览器这个程序就需要通过网络IO获取新浪的网页。浏览器首先会发送数据给新浪服务器，告诉它我想要首页的HTML，这个动作是往外发数据，叫Output，随后新浪服务器把网页发过来，这个动作是从外面接收数据，叫Input。所以，通常，程序完成IO操作会有Input和Output两个数据流。当然也有只用一个的情况，比如，从磁盘读取文件到内存，就只有Input操作，反过来，把数据写到磁盘文件里，就只是一个Output操作。

IO编程中，Stream（流）是一个很重要的概念，可以把流想象成一个水管，数据就是水管里的水，但是只能单向流动。Input Stream就是数据从外面（磁盘、网络）流进内存，Output Stream就是数据从内存流到外面去。对于浏览网页来说，浏览器和新浪服务器之间至少需要建立两根水管，才可以既能发数据，又能收数据。

由于CPU和内存的速度远远高于外设的速度，所以，在IO编程中，就存在速度严重不匹配的问题。举个例子来说，比如要把100M的数据写入磁盘，CPU输出100M的数据只需要0.01秒，可是磁盘要接收这100M数据可能需要10秒，怎么办呢？有两种办法：

第一种是CPU等着，也就是程序暂停执行后续代码，等100M的数据在10秒后写入磁盘，再接着往下执行，这种模式称为同步IO；

另一种方法是CPU不等待，只是告诉磁盘，“您老慢慢写，不着急，我接着干别的事去了”，于是，后续代码可以立刻接着执行，这种模式称为异步IO。

同步和异步的区别就在于是否等待IO执行的结果。好比你去麦当劳点餐，你说“来个汉堡”，服务员告诉你，对不起，汉堡要现做，需要等5分钟，于是你站在收银台前面等了5分钟，拿到汉堡再去逛商场，这是同步IO。

你说“来个汉堡”，服务员告诉你，汉堡需要等5分钟，你可以先去逛商场，等做好了，我们再通知你，这样你可以立刻去干别的事情（逛商场），这是异步IO。

很明显，使用异步IO来编写程序性能会远远高于同步IO，但是异步IO的缺点是编程模型复杂。想想看，你得知道什么时候通知你“汉堡做好了”，而通知你的方法也各不相同。如果是服务员跑过来找到你，这是回调模式，如果服务员发短信通知你，你就得不停地检查手机，这是轮询模式。总之，异步IO的复杂度远远高于同步IO。

# 操作文件和目录

In [2]:
import os

os.name  # 操作系统类型

'posix'

In [4]:
os.uname()  # 获取详细的系统信息

posix.uname_result(sysname='Linux', nodename='book', release='4.15.0-36-generic', version='#39-Ubuntu SMP Mon Sep 24 16:19:09 UTC 2018', machine='x86_64')

## 环境变量


In [5]:
os.environ # 在操作系统中定义的环境变量， 全部保存在os.environ这个变量中

environ({'CLUTTER_IM_MODULE': 'xim', 'LD_LIBRARY_PATH': '/opt/ros/melodic/lib', 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.wim=01;31:*.swm=01;31:*.dwm=01;31:*.esd=01;31:*.jpg=01;35:*.jpeg=01;35:*.mjpg=01;35:*.mjpeg=01;35:*.gif=01;35:*.bmp=01;35:*.pbm=01;35:*.pgm=01;35:*.ppm=01;35:*.tga=01;35:*.xbm=01;35:*.xpm=01;35:*.tif=01;35:*.tiff=01;35:*.png=01;35:*.sv

In [6]:
os.environ.get('PATH') # 要获取某个环境变量的值， 可以调用os.environ.get('key'):

'/opt/ros/melodic/bin:/home/bo/java/java/bin:/home/bo/anaconda3/bin:/home/bo/bin:/home/bo/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin'

In [7]:
os.environ.get('x', 'default')

'default'

## 操作文件和目录

操作文件和目录的函数一部分放在os模块中，一部分放在os.path模块中， 这一点要注意一下。查看、创建、删除目录可以这么调用：

In [8]:
# 查看当前目录的绝对路径
os.path.abspath('.')

'/home/bo/Desktop/sundries/python_recall'

In [9]:
# 在某个目录下创建一个新目录， 首先把新目录的完整路径表示出来
os.path.join('/home/bo/Desktop/sundries/python_recall', 'testdir')

'/home/bo/Desktop/sundries/python_recall/testdir'

In [10]:
# 然后创建一个目录
os.mkdir('/home/bo/Desktop/sundries/python_recall/testdir')

In [11]:
# 删除一个目录
os.rmdir('/home/bo/Desktop/sundries/python_recall/testdir')

把两个路径合成一个时，不要直接拼字符串，而要通过os.path.join()函数，这样可以正确处理不同操作系统的路径分隔符。在Linux/Unix/Mac下，os.path.join()返回这样的字符串：

In [12]:
os.path.split('/home/bo/Desktop/sundries/python_recall')

('/home/bo/Desktop/sundries', 'python_recall')

In [14]:
os.path.splitext('/home/bo/Desktop/sundries/python_recall') # os.path.splitext()可以直接获得文件的扩展名。很方便

('/home/bo/Desktop/sundries/python_recall', '')

In [16]:
# 最后看看如何利用Python的特性来过滤文件。比如我们要列出当前目录下的所有目录，只需要一行代码：
[x for x in os.listdir('.')]

['函数式编程.ipynb',
 '面向对象的高级编程.ipynb',
 'Advanced_feature.ipynb',
 '.ipynb_checkpoints',
 'IO.ipynb',
 '面向对象编程.ipynb',
 'recursive.ipynb',
 '错误处理.ipynb']

In [18]:
# 过滤文件
[x for x in os.listdir('.') if os.path.isfile(x) and os.path.splitext(x)[1] == '.ipynb']

['函数式编程.ipynb',
 '面向对象的高级编程.ipynb',
 'Advanced_feature.ipynb',
 'IO.ipynb',
 '面向对象编程.ipynb',
 'recursive.ipynb',
 '错误处理.ipynb']

# 序列化

python 提供了pickle 模块来实现序列化。

首先，先尝试把一个对象序列化并写入文件：

In [21]:
import pickle

d = dict(name='Bob', age=20, score=88)
pickle.dumps(d) 

b'\x80\x03}q\x00(X\x04\x00\x00\x00nameq\x01X\x03\x00\x00\x00Bobq\x02X\x03\x00\x00\x00ageq\x03K\x14X\x05\x00\x00\x00scoreq\x04KXu.'

pickle.dumps()方法把任意对象序列化成一个bytes，然后，就可以把这个bytes写入文件。或者用另一个方法pickle.dump()直接把对象序列化后写入一个file-like Object：

In [40]:
d

{'age': 20, 'name': 'Bob', 'score': 88}

In [41]:
f = open('dump.txt', 'wb')
pickle.dump(d, f)
f.close()
# 看看写入的dump.txt文件，一堆乱七八糟的内容，这些都是Python保存的对象内部信息。

In [42]:
# 当我们要把对象从磁盘读到内存时，可以先把内容读到一个bytes，
# 然后用pickle.loads()方法反序列化出对象，也可以直接用pickle.load()方法从一个file-like Object中直接反序列化出对象。
# 我们打开另一个Python命令行来反序列化刚才保存的对象：

f = open('dump.txt', 'rb')
d = pickle.load(f)
f.close()
d

{'age': 20, 'name': 'Bob', 'score': 88}

# JSON
如果我们要在不同的编程语言之间传递对象，就必须把对象序列化为标准格式，比如XML，但更好的方法是序列化为JSON，因为JSON表示出来就是一个字符串，可以被所有语言读取，也可以方便地存储到磁盘或者通过网络传输。JSON不仅是标准格式，并且比XML更快，而且可以直接在Web页面中读取，非常方便。


In [43]:
import json
d = dict(name='Bob', age = 20, score=88)
json.dumps(d)

'{"name": "Bob", "age": 20, "score": 88}'